# Equation Solving Tools

We can solve systems of equations exactly using sympy’s `solve` function. This is usually done using what is known as the residual form. The residual is simply the difference between the LHS and RHS of an equation, or put another way, we rewrite our equations to be equal to zero:

$$ \begin{align}
x + y &= z \\
\therefore x + y - z &= 0
\end{align}
$$

In [ ]:
import sympy
sympy.init_printing()
%matplotlib inline

In [ ]:
x, y, z = sympy.symbols('x, y, z')
sympy.solve(x + y - z, z)

In [ ]:
equations = [x + y - z,
            2*x+y +z +2,
            x - y -z +2,]
unknowns = [x, y, z]

In [ ]:
solution = sympy.solve(equations, unknowns)
solution

In [ ]:
%%timeit
sympy.solve(equations, unknowns)

In [ ]:
solution[x]

In [ ]:
# floating point version
solution[x].n()

### 9.2 Special Case: Linear System

For linear systems like the one above, we can solve very efficiently using matrix algebra. The system of equations can be rewritten in matrix form:

$$ Ax = b$$

In [ ]:
equations

In [ ]:
A = sympy.Matrix([[1, 1, -1],
                  [2, 1, 1],
                  [1, -1, -1]])
b = sympy.Matrix([[0, -2, -2]]).T

In [ ]:
A.solve(b)

In [ ]:
%%time
A.solve(b)

In [ ]:
import numpy

In [ ]:
A = numpy.matrix([[1, 1, -1],
                  [2, 1, 1],
                  [1, -1, -1]])
b = numpy.matrix([[0, -2, 2]]).T

In [ ]:
numpy.linalg.solve(A, b)

In [ ]:
%%time
numpy.linalg.solve(A, b)

In [ ]:
N = 30
bigA = numpy.random.random((N, N))

In [ ]:
bigB = numpy.random.random((N,))

In [ ]:
%%timeit
numpy.linalg.solve(bigA, bigB)

In [ ]:
bigsymbolicA = sympy.Matrix(bigA)

In [ ]:
bigsymbolicb = sympy.Matrix(bigB)

In [ ]:
%%timeit
bigsymbolicA.solve(bigsymbolicb)

## Nonlinear Equations

In [ ]:
x, y = sympy.symbols('x,y')

In [ ]:
sympy.solve([x + sympy.log(y), y**2 - 1], [x, y])

In [ ]:
x, y = sympy.symbols('x, y', real=True)

In [ ]:
sympy.solve([x + sympy.log(y), y**2 - 1], [x, y])

### Numeric Root Finding

In [ ]:
unsolvable = x + sympy.cos(x) + sympy.log(x)

In [ ]:
sympy.plot(unsolvable)

In [ ]:
sympy.plot(unsolvable, (x, 0.1, 1))

In [ ]:
sympy.nsolve(unsolvable, x, 0.3)

In [ ]:
import scipy.optimize

In [ ]:
plus_two = lambda x: x+2

In [ ]:
plus_two(2)

In [ ]:
def plus_two(x):
    return x + 2

In [ ]:
unsolvable_numeric = sympy.lambdify(x, unsolvable)

In [ ]:
unsolvable_numeric(0.3)

In [ ]:
scipy.optimize.fsolve(unsolvable_numeric, 0.1)

In [ ]:
def multiple_equations(unknowns):
    x, y = unknowns
    return [x + y -1 ,
            x - y]

In [ ]:
multiple_equations([1, 2])

In [ ]:
first_guess =  [1, 1]
scipy.optimize.fsolve(multiple_equations, first_guess)

In [ ]:
from IPython import display

In [ ]:
display.Image("./img/tanksystem.png")

$$ \begin{align}
F_{out} &= kh\\
\frac{\mathrm{d}h}{\mathrm{d}t} &= \frac{1}{A}\left(F_{in} - F_{out}\right)\\
\end{align}
$$

### Analytic Solution

In [ ]:
h = sympy.Function('h') # This is how to specify an unknown function in sympy
t = sympy.Symbol('t', positive = True)

In [ ]:
Fin = 2
K = 1
A = 1  

In [ ]:
Fout = K*h(t)
Fout

In [ ]:
de = h(t).diff(t) - 1/A*(Fin - Fout)
de

In [ ]:
solution = sympy.dsolve(de)
solution

In [ ]:
C1 = solution.rhs.args[1].args[0]

In [ ]:
h0 = 1

In [ ]:
constants = sympy.solve(solution.rhs.subs({t: 0}) - h0, C1)
constants

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
sympy.plot(solution.rhs.subs({C1: constants[0]}), (t, 0, 10))

### Numeric Solution

In [ ]:
import scipy.integrate
import numpy as np

In [ ]:
Fin = 2

In [ ]:
def dhdt(t, h):
    """Function returning derivative of h - note it takes t and has arguments"""
    Fout = K *h
    return 1/A*(Fin - Fout)


In [ ]:
tspan = (0, 10)

In [ ]:
sol = scipy.integrate.solve_ivp(dhdt, tspan, [h0])

In [ ]:
tsmooth = np.linspace(0, 10, 1000)
hnanalytic = 2 - np.exp(-tsmooth)

In [ ]:
plt.plot(sol.t, sol.y.T, 'o-', label = 'solve_ivp solution')
plt.plot(tsmooth, hnanalytic, label = 'Analytic Solution')
plt.legend()

In [ ]:
sol = scipy.integrate.solve_ivp(dhdt, tspan, [h0], t_eval = tsmooth)

In [ ]:
plt.plot(tsmooth, sol.y.T)
plt.plot(tsmooth, hnanalytic, '--')

In [ ]:
import scipy.integrate

In [ ]:
def Fin(t):
    """A step which starts at t=2"""
    if t < 2:
        return 1
    else:
        return 2

In [ ]:
def dhdt(t, h):
    Fout = K * h
    return 1/A*(Fin(t) - Fout)

In [ ]:
tspan = (0, 10)

In [ ]:
sol = scipy.integrate.solve_ivp(dhdt, tspan, [h0])
smoothsol = scipy.integrate.solve_ivp(dhdt, tspan, [h0], t_eval=tsmooth)

In [ ]:
plt.plot(sol.t, sol.y.T, 'o')
plt.plot(smoothsol.t, smoothsol.y.T)

In [ ]:
sol = scipy.integrate.solve_ivp(dhdt, tspan, [h0], max_step = 0.1)
smoothsol = scipy.integrate.solve_ivp(dhdt, tspan, [h0], t_eval=tsmooth, max_step=0.1)

In [ ]:
plt.plot(sol.t, sol.y.T, 'o')
plt.plot(smoothsol.t, smoothsol.y.T)

In [ ]:
%%timeit
sol = scipy.integrate.solve_ivp(dhdt, tspan, [h0])

In [ ]:
%%timeit
sol = scipy.integrate.solve_ivp(dhdt, tspan, [h0], max_step = 0.1)

### A note about odeint

In [ ]:
def odeintdhdt(h, t):
    """Odeint expects a fucntion with the arguments reversed from solve_ivp"""
    return dhdt(t, h)

In [ ]:
odeinth = scipy.integrate.odeint(odeintdhdt, h0, tsmooth)

In [ ]:
plt.plot(sol.t, sol.y.T, 'o')
plt.plot(tsmooth, odeinth)